# Prepare data

In [1]:
# add library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline   
import seaborn as sns
import warnings
import matplotlib as mpl
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier


In [4]:
data_file = './data/GSE66099.csv'
degs_file = './gene_diff_data.csv'
# data pre-processing
norm_filter = MinMaxScaler()

label = 'Died'
num_features = 10

DEGs = pd.read_csv(degs_file)
degs_gene_id = DEGs["X"]

data = pd.read_csv(data_file)
degs_gene_data = data[degs_gene_id]
degs_gene_data.loc[:,"label"]  = data[label]
degs_gene_data

C:\Users\vuduc\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\vuduc\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,SLC39A8,RHAG,DDIT4,MPO,RRM2,CCL3,TGFBI,MAFF,TYMS,ENPP2,...,FEN1,AURKA,FRAT1,SNRPG,KIF2C,POLE2,UBE2C,AURKB,CENPF,label
0,7.837760,4.112866,8.046405,2.324069,4.759059,3.140721,4.143039,8.578116,5.230842,3.862937,...,4.062130,3.009849,8.785775,9.787393,2.240854,2.622261,4.787508,3.002504,2.246339,1
1,7.751983,1.485795,8.971049,5.258546,7.684430,2.811141,4.977650,4.715507,7.567845,3.212268,...,4.651380,4.084429,8.823993,9.877403,3.083158,2.470562,5.049623,2.667229,3.301098,0
2,10.279053,1.925620,8.712714,1.856427,5.890788,5.947866,4.036440,9.109963,6.066830,3.086101,...,3.554817,2.921564,8.135534,10.673182,2.399208,2.270243,4.136699,3.181530,2.663032,1
3,9.601609,5.888388,8.527441,5.750055,8.206514,5.134266,4.616064,5.156271,8.057016,2.684719,...,5.330526,4.034944,7.884369,9.025149,3.205451,2.998327,5.846744,3.883344,3.662963,1
4,7.785195,4.846312,9.301229,8.695507,10.032503,6.555199,6.542890,8.041716,9.813448,7.125344,...,6.783366,6.314857,8.603358,10.328848,4.966912,4.004817,6.864789,5.244933,4.199692,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,7.181331,2.365467,8.184828,6.181315,7.871825,2.929050,7.998090,4.431776,7.034406,3.780909,...,4.477342,4.055716,8.839066,9.910092,2.871094,3.078237,3.940554,3.216972,3.462929,0
195,5.251613,2.714796,6.427088,3.523980,7.221401,2.154800,4.979873,7.541867,7.252205,2.629904,...,5.162441,3.987030,9.600255,6.991776,2.501947,2.859321,4.154692,3.430309,3.199781,0
196,5.905524,8.074482,6.259182,2.492298,5.146876,1.794615,8.446067,8.769124,4.142922,2.246056,...,4.168839,4.630949,7.698138,9.373841,2.188718,2.145200,4.212061,2.516086,2.280285,0
197,4.786756,2.279402,5.955421,8.199064,8.008226,3.352024,9.477510,5.653967,7.454457,2.954322,...,4.676477,4.053300,8.976000,6.692781,2.799296,2.676570,4.434418,3.215429,3.070606,0


In [46]:
degs_gene_data.to_csv("108_gene_data.csv")

In [5]:
# split data 7/3
from sklearn.model_selection import train_test_split

# y = degs_gene_data.iloc[:,108].values
X, y = degs_gene_data.iloc[:,0:108].values, degs_gene_data.iloc[:,108].values

In [6]:
# normalize data:
def nml(gene_data):
    normalize_data = (gene_data - min(gene_data))/(max(gene_data) - min(gene_data))
    return normalize_data
for i in range(108):
    X[:,i] = nml(X[:,i])

In [10]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1234,
                                                   shuffle=True, stratify=y)

In [11]:
y_test

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [12]:
from sklearn import metrics

rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

print('Training accuracy:', np.mean(rf.predict(X_train) == y_train)*100)
print('Test accuracy:', np.mean(rf.predict(X_test) == y_test)*100)

Training accuracy: 100.0
Test accuracy: 90.0


In [13]:
def evaluate(y_pred, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(y_test)):
        if (y_pred[i]==1 and y_test[i]==1):
            tp = tp + 1
        elif (y_pred[i]==0 and y_test[i]==0):
            tn = tn + 1
        elif (y_pred[i]==1 and y_test[i]==0):
            fp = fp + 1
        else:
            fn = fn + 1
    sn = tp/(tp+fn)
    sp = tn/(tn+fp)
    fpr = fp/(tn+fp)
    acc = (tp+tn)/(tp+fp+tn+fn)
    M = {
        "sn":sn,
        "sp":sp,
        "acc":acc,
        "tp":tp,
        "tn":tn,
        "fp":fp,
        "fn":fn,
        "fpr":fpr
    }
    return M   
      
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = rf.predict(X_test)
metric = evaluate(y_pred, y_test)


In [14]:
metric

{'sn': 0.42857142857142855,
 'sp': 0.9767441860465116,
 'acc': 0.9,
 'tp': 6,
 'tn': 84,
 'fp': 2,
 'fn': 8,
 'fpr': 0.023255813953488372}

# SFS with score

In [39]:
one_arr = np.ones(109)
one_arr.shape

(109,)

In [40]:
# SFS score
# Set random seed
seed = 8
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size = 0.3, random_state=1234,
                                                   shuffle=True, stratify=y_train)

# model = SVC(kernel='rbf',probability=True)
model = RandomForestClassifier(n_estimators=100, random_state=seed)
total_score = []
# model = LogisticRegression(penalty="l2", random_state=0)
for i in range(108):
    c = pd.DataFrame(X_train2[:,i])
    d = pd.DataFrame(X_test2[:,i])
#     c = pd.DataFrame((X_train2.values)[:,i])
#     d = pd.DataFrame((X_test2.values)[:,i])
    model.fit(c, y_train2)
    y_pred2 = model.predict(d)
    metric = evaluate(y_pred2, y_test2)
    total_score.append(metric["acc"])

In [41]:
(X_train2).shape

(69, 108)

In [42]:
max_ind = np.argmax(total_score)
max_ind

43

In [43]:
score = pd.DataFrame(total_score)
score

,0
0,0.800000
1,0.800000
2,0.800000
3,0.633333
4,0.800000
...,...
103,0.733333
104,0.800000
105,0.733333
106,0.833333


In [44]:
gene_score = pd.DataFrame(degs_gene_id)
gene_score["score"] = score

In [45]:
gene_score.to_csv("gene_score.csv")

In [22]:
gene = []
for i in degs_gene_id:
    gene.append(i)
gene_rank = []
gene_score_ind = []
for i in range(108):
    max_ind = np.argmax(total_score)
    gene_rank.append(gene[max_ind])
    gene_score_ind.append(max_ind)
    total_score[max_ind] = 0

In [23]:
a = gene_score_ind
total_comb = [None]*108
for i in range(108):
    arr = []
    for j in range(0, i + 1):
        arr.append(a[j])
    total_comb[i] = arr
    

In [24]:
for val in total_comb:
    print(val)

[43]
[43, 48]
[43, 48, 64]
[43, 48, 64, 5]
[43, 48, 64, 5, 8]
[43, 48, 64, 5, 8, 39]
[43, 48, 64, 5, 8, 39, 54]
[43, 48, 64, 5, 8, 39, 54, 65]
[43, 48, 64, 5, 8, 39, 54, 65, 82]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50, 66]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50, 66, 69]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50, 66, 69, 72]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50, 66, 69, 72, 93]
[43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45, 50, 66, 69, 72, 93, 95]
[43

In [25]:
# get combination score
combination_value = []
# model2 = SVC(kernel='rbf',probability=True)
model2 = RandomForestClassifier(n_estimators=100, random_state=0)
# model2 = LogisticRegression(penalty="l2", random_state=0)
# model2 = KNeighborsClassifier(n_neighbors=11, p=2, n_jobs=-1)
comb_score = []
for val in total_comb:
    c = pd.DataFrame((X_train2)[:,val])
    d = pd.DataFrame((X_test2)[:,val])
    model2.fit(c, y_train2)
    y_pred2 = model2.predict(d)
    metric = evaluate(y_pred2, y_test2)
    combination_value.append({
        "com":val,
        "value":metric["acc"]
    })
    comb_score.append(metric["acc"])

In [26]:
len(combination_value)

108

In [27]:
combination_value

[{'com': [43], 'value': 0.9333333333333333},
 {'com': [43, 48], 'value': 0.8666666666666667},
 {'com': [43, 48, 64], 'value': 0.9},
 {'com': [43, 48, 64, 5], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82], 'value': 0.8666666666666667},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32], 'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35],
  'value': 0.9},
 {'com': [43, 48, 64, 5, 8, 39, 54, 65, 82, 84, 91, 92, 17, 32, 35, 45],
  'value': 0.9},
 {'com': [43, 48

In [28]:
(pd.DataFrame(combination_value)).to_csv("combination_value2.csv")

In [29]:
comb_score

[0.9333333333333333,
 0.8666666666666667,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.8666666666666667,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9333333333333333,
 0.9,
 0.9,
 0.9333333333333333,
 0.9,
 0.9,
 0.9333333333333333,
 0.9,
 0.9,
 0.9,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9,
 0.9,
 0.9333333333333333,
 0.9333333333333333,
 0.9,
 0.9333333333333333,
 0.9,
 0.8666666666666667,
 0.9,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9,
 0.9,
 0.9333333333333333,
 0.9,
 0.9,
 0.9,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9,
 0.9,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.8666666666666667,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333333333333,
 0.9333333

In [30]:
comb_ind = np.argmax(comb_score)

# evaluate model with gene_marker

In [31]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from rdea.utils.help_functions import model_based_relevance, auc_relevance, load_DEGs


import matplotlib.pyplot as plt
%matplotlib inline   
import seaborn as sns
import warnings

# Comment these lines if you are not using Mac OS
import matplotlib as mpl
# Set random seed
seed = 8
np.random.seed(seed)

# Input & parameter

In [32]:
# ========== INPUTS ==========
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# in and out files
data_file = './data/GSE66099.csv'
markers_file = './results/sepsis/fs_scores.csv'

out_file = './results/sepsis/fs_scores.csv'
is_save_results = True

# data pre-processing
is_normalize_samples = False
is_normalize_columns = True
norm_filter = MinMaxScaler()

# experiment
num_folds = 5

label = 'Died'
# Set random seed
seed = 8
np.random.seed(seed)

# classifiers
models = [
  ('SVM-RBF',SVC(kernel='rbf',probability=True)),
  ('RF', RandomForestClassifier(n_estimators=100,  random_state=seed)),
  ('LR', SGDClassifier(loss="log", penalty="l2", random_state=seed)),
  ('KNN', KNeighborsClassifier(n_neighbors=15, p=2, n_jobs=-1)),
  ('Bagging', BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=seed), n_estimators=10, random_state=seed)),  
  ('Boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=seed))      
 ]



In [ ]:

gene_selected = (degs_gene_id.values)[(combination_value[20])['com']]
features = gene_selected
markers_sets = []
for i in range(1, len(features) + 1):
    markers_sets.append(('top_'+str(i),features[:i]))

In [ ]:
# Load data 
# df = pd.read_csv(data_file)
# print('Shape of {0} is {1}'.format(data_file, df.shape))
# new_y = df[label]
new_y = y_test

In [ ]:
X = X_train[:, (combination_value[20])['com']]
X.shape

# Searching optimal algorithms

In [33]:
def evaluate2(y_pred, y_test, y_probs):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, y_probs)
    for i in range(len(y_test)):
        if (y_pred[i]==1 and y_test[i]==1):
            tp = tp + 1
        elif (y_pred[i]==0 and y_test[i]==0):
            tn = tn + 1
        elif (y_pred[i]==1 and y_test[i]==0):
            fp = fp + 1
        else:
            fn = fn + 1
    sn = tp/(tp+fn)
    sp = tn/(tn+fp)
    fpr = fp/(tn+fp)
    acc = (tp+tn)/(tp+fp+tn+fn)
    return  np.array([acc, sn, sp, auc])

In [34]:
from sklearn.model_selection import StratifiedKFold, KFold
from scipy import interp
from sklearn.metrics import *
def do_cross_validation_for_sing_model(X_comb, new_y, model, num_folds=5, random_state = 0, n_runs = 5):
   
    res = np.zeros(shape=(num_folds * n_runs, 4)) # 5 is the number of metrics
    count = 0
    for cv_run in range(n_runs):
        skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
        for train_index, test_index in skf.split(X_comb, new_y):
            X_train2, X_test2 = X_comb[train_index], X_comb[test_index]
            y_train2, y_test2 = new_y[train_index], new_y[test_index]
            
            train_filtered = X_train2
            test_filtered = X_test2

            model.fit(train_filtered, y_train2)
            Y_probs = model.predict_proba(test_filtered)[:, 1]
            Y_pred = model.predict(test_filtered)

            res[count, :] = evaluate2(Y_pred, y_test2, Y_probs)
            count += 1
            
    return res

In [35]:
import math
def std_mean_cal(res):
    means = [None]*4
    stds = [None]*4
    for i in range(4):
        mean = round(np.mean(res[:,i]),2)
        means[i] = mean
#         std = sum(np.square(res[:,i] - means[i]))/25
        stds[i] = round(np.std(res[:,i]),2)
    M = [means, stds]
    return M


In [ ]:
print('Performance reported using:\t ACC\tSn\tSp\tMCC\tAUC')

for model in models:
    count = 0
    total = []
    for comb in total_comb:
        X_comb = pd.DataFrame((X_test)[:,comb])
        num_runs = 5
        res = do_cross_validation_for_sing_model(X_comb.values, y_test, model=model[1], num_folds=5, n_runs=5)
        result = std_mean_cal(res)
        count += 1
        mean_std = {
            "Comb": comb,
            "ACC": str((result[0])[0]) + "+-" + str((result[1])[0]),
            "Sn": str((result[0])[1]) + "+-" + str((result[1])[1]),
            "Sp": str((result[0])[2]) + "+-" + str((result[1])[2]),
            "AUC": str((result[0])[3]) + "+-" + str((result[1])[3])
        }
        total.append(mean_std)
        print(res)
        print(count)
        print('---------------------------------------------------------------')
    file_name = model[0] + "- result2.csv"
    (pd.DataFrame(total)).to_csv(file_name)
        



# 5 fold cross validation in validation set

In [36]:
seed = 8
models2 = [
  ('SVM-RBF',SVC(kernel='rbf',probability=True)),
  ('RF', RandomForestClassifier(n_estimators=100,  random_state=seed)),
  ('LR', SGDClassifier(loss="log", penalty="l2", random_state=seed)),
  ('KNN', KNeighborsClassifier(n_neighbors=15, p=2, n_jobs=-1)),
  ('Bagging', BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=seed), n_estimators=10, random_state=seed)),  
  ('Boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=seed))      
 ]


In [37]:
print('Performance reported using:\t ACC\tSn\tSp\tMCC\tAUC')

for model in models2:
    count = 0
    total = []
    for comb in total_comb:
        X_comb = pd.DataFrame((X_test)[:,comb])
        num_runs = 10
        res = do_cross_validation_for_sing_model(X_comb.values, y_test, model=model[1], num_folds=5, n_runs=5)
        result = std_mean_cal(res)
        count += 1
        mean_std = {
            "Comb": comb,
            "ACC": str((result[0])[0]) + "+-" + str((result[1])[0]),
            "Sn": str((result[0])[1]) + "+-" + str((result[1])[1]),
            "Sp": str((result[0])[2]) + "+-" + str((result[1])[2]),
            "AUC": str((result[0])[3]) + "+-" + str((result[1])[3])
        }
        total.append(mean_std)
        print(res)
        print(count)
        print('---------------------------------------------------------------')
    file_name = model[0] + "- result-test-10runs.csv"
    (pd.DataFrame(total)).to_csv(file_name)
        

Performance reported using:	 ACC	Sn	Sp	MCC	AUC
[[0.85       0.         1.         0.88235294]
 [0.85       0.         1.         0.62745098]
 [0.85       0.         1.         0.33333333]
 [0.85       0.         1.         0.58823529]
 [0.9        0.         1.         0.91666667]
 [0.85       0.         1.         0.37254902]
 [0.85       0.         1.         0.31372549]
 [0.85       0.         1.         0.35294118]
 [0.8        0.         0.94117647 0.43137255]
 [0.9        0.         1.         0.91666667]
 [0.85       0.         1.         0.60784314]
 [0.85       0.         1.         0.2745098 ]
 [0.8        0.         0.94117647 0.43137255]
 [0.85       0.         1.         0.98039216]
 [0.9        0.         1.         0.19444444]
 [0.85       0.         1.         0.92156863]
 [0.85       0.         1.         0.56862745]
 [0.8        0.         0.94117647 0.31372549]
 [0.85       0.         1.         0.64705882]
 [0.9        0.         1.         0.94444444]
 [0.85       

[[0.85       0.         1.         0.84313725]
 [0.85       0.         1.         0.47058824]
 [0.85       0.         1.         0.76470588]
 [0.85       0.         1.         0.92156863]
 [0.9        0.         1.         0.52777778]
 [0.85       0.         1.         0.66666667]
 [0.85       0.         1.         0.50980392]
 [0.85       0.         1.         0.68627451]
 [0.85       0.         1.         0.58823529]
 [0.9        0.         1.         0.83333333]
 [0.85       0.         1.         0.56862745]
 [0.85       0.         1.         0.76470588]
 [0.85       0.         1.         0.92156863]
 [0.85       0.         1.         0.54901961]
 [0.9        0.         1.         0.61111111]
 [0.85       0.         1.         0.92156863]
 [0.8        0.         0.94117647 0.78431373]
 [0.85       0.         1.         0.49019608]
 [0.85       0.         1.         0.62745098]
 [0.9        0.         1.         0.5       ]
 [0.85       0.         1.         0.68627451]
 [0.85       

[[0.85       0.         1.         0.74509804]
 [0.85       0.         1.         0.64705882]
 [0.85       0.         1.         0.60784314]
 [0.8        0.         0.94117647 0.80392157]
 [0.9        0.         1.         0.41666667]
 [0.85       0.         1.         0.90196078]
 [0.85       0.         1.         0.68627451]
 [0.85       0.         1.         0.68627451]
 [0.85       0.         1.         0.39215686]
 [0.9        0.         1.         0.55555556]
 [0.85       0.         1.         0.74509804]
 [0.85       0.         1.         0.90196078]
 [0.85       0.         1.         0.64705882]
 [0.85       0.         1.         0.60784314]
 [0.9        0.         1.         0.69444444]
 [0.85       0.         1.         0.88235294]
 [0.85       0.         1.         0.43137255]
 [0.85       0.         1.         0.80392157]
 [0.85       0.         1.         0.74509804]
 [0.85       0.         0.94444444 0.83333333]
 [0.85       0.         1.         0.41176471]
 [0.85       

[[0.85       0.         1.         0.82352941]
 [0.85       0.         1.         0.74509804]
 [0.85       0.         1.         0.84313725]
 [0.85       0.         1.         0.88235294]
 [0.9        0.         1.         0.63888889]
 [0.85       0.         1.         0.70588235]
 [0.8        0.         0.94117647 0.80392157]
 [0.85       0.         1.         0.96078431]
 [0.85       0.         1.         0.76470588]
 [0.9        0.         1.         0.58333333]
 [0.85       0.         1.         0.68627451]
 [0.85       0.         1.         0.58823529]
 [0.85       0.         1.         0.90196078]
 [0.85       0.         1.         0.88235294]
 [0.9        0.         1.         0.86111111]
 [0.85       0.         1.         0.78431373]
 [0.85       0.         1.         0.70588235]
 [0.85       0.         1.         0.78431373]
 [0.85       0.         1.         0.64705882]
 [0.9        0.         1.         0.72222222]
 [0.85       0.         1.         0.92156863]
 [0.85       

[[0.85       0.         1.         0.66666667]
 [0.85       0.         1.         0.74509804]
 [0.85       0.         1.         0.54901961]
 [0.85       0.         1.         0.43137255]
 [0.9        0.         1.         0.94444444]
 [0.85       0.         1.         0.8627451 ]
 [0.85       0.         1.         0.62745098]
 [0.85       0.         1.         0.35294118]
 [0.85       0.         1.         0.92156863]
 [0.9        0.         1.         0.05555556]
 [0.85       0.         1.         0.68627451]
 [0.85       0.         1.         0.90196078]
 [0.85       0.         1.         0.62745098]
 [0.85       0.         1.         0.82352941]
 [0.9        0.         1.         0.36111111]
 [0.85       0.         1.         0.76470588]
 [0.85       0.         1.         0.74509804]
 [0.85       0.         1.         0.60784314]
 [0.85       0.         1.         0.78431373]
 [0.9        0.         1.         0.72222222]
 [0.85       0.         1.         0.96078431]
 [0.85       

...

[[0.95       0.66666667 1.         0.78431373]
 [0.75       0.33333333 0.82352941 0.56862745]
 [0.7        0.33333333 0.76470588 0.42156863]
 [0.8        0.         0.94117647 0.80392157]
 [0.9        0.5        0.94444444 0.86111111]
 [0.7        0.33333333 0.76470588 0.45098039]
 [0.5        0.66666667 0.47058824 0.52941176]
 [0.9        0.33333333 1.         0.58823529]
 [0.8        0.66666667 0.82352941 0.7745098 ]
 [0.7        0.         0.77777778 0.43055556]
 [0.85       0.66666667 0.88235294 0.88235294]
 [0.8        0.33333333 0.88235294 0.64705882]
 [0.8        0.66666667 0.82352941 0.88235294]
 [0.65       0.33333333 0.70588235 0.33333333]
 [0.85       0.         0.94444444 0.        ]
 [0.85       1.         0.82352941 1.        ]
 [0.85       0.33333333 0.94117647 0.70588235]
 [0.65       0.         0.76470588 0.52941176]
 [0.75       0.         0.88235294 0.47058824]
 [0.95       0.5        1.         0.88888889]
 [0.7        0.         0.82352941 0.80392157]
 [0.8        

[[0.65       0.33333333 0.70588235 0.44117647]
 [0.8        0.         0.94117647 0.7254902 ]
 [0.7        0.         0.82352941 0.44117647]
 [0.85       1.         0.82352941 0.92156863]
 [0.85       0.5        0.88888889 0.58333333]
 [0.8        0.33333333 0.88235294 0.68627451]
 [0.8        0.33333333 0.88235294 0.71568627]
 [0.75       0.33333333 0.82352941 0.45098039]
 [0.5        0.33333333 0.52941176 0.47058824]
 [0.9        0.         1.         0.5       ]
 [0.95       0.66666667 1.         0.68627451]
 [0.65       0.66666667 0.64705882 0.61764706]
 [0.65       0.         0.76470588 0.35294118]
 [0.85       0.33333333 0.94117647 0.78431373]
 [0.7        0.5        0.72222222 0.66666667]
 [0.75       0.         0.88235294 0.5       ]
 [0.65       0.         0.76470588 0.37254902]
 [0.85       0.33333333 0.94117647 0.92156863]
 [0.75       0.33333333 0.82352941 0.74509804]
 [0.9        1.         0.88888889 0.94444444]
 [0.75       0.33333333 0.82352941 0.58823529]
 [0.8        

[[0.85       1.         0.82352941 0.96078431]
 [0.65       0.         0.76470588 0.39215686]
 [0.7        0.33333333 0.76470588 0.57843137]
 [0.85       0.66666667 0.88235294 0.76470588]
 [0.8        0.         0.88888889 0.63888889]
 [0.85       0.33333333 0.94117647 0.76470588]
 [0.6        0.33333333 0.64705882 0.55882353]
 [0.65       0.         0.76470588 0.37254902]
 [0.65       0.33333333 0.70588235 0.71568627]
 [0.75       0.         0.83333333 0.61111111]
 [0.75       0.         0.88235294 0.51960784]
 [0.85       0.33333333 0.94117647 0.54901961]
 [0.7        0.33333333 0.76470588 0.65686275]
 [0.65       0.         0.76470588 0.40196078]
 [0.85       0.5        0.88888889 0.94444444]
 [0.8        0.33333333 0.88235294 0.78431373]
 [0.75       0.         0.88235294 0.42156863]
 [0.85       0.         1.         0.35294118]
 [0.95       1.         0.94117647 0.98039216]
 [0.75       0.         0.83333333 0.58333333]
 [0.85       0.33333333 0.94117647 0.76470588]
 [0.8        

[[0.7        0.         0.82352941 0.66666667]
 [0.9        0.33333333 1.         0.37254902]
 [0.75       1.         0.70588235 0.78431373]
 [0.85       0.33333333 0.94117647 0.49019608]
 [0.85       0.         0.94444444 0.22222222]
 [0.8        0.         0.94117647 0.40196078]
 [0.85       0.33333333 0.94117647 0.56862745]
 [0.75       0.         0.88235294 0.33333333]
 [0.65       0.66666667 0.64705882 0.61764706]
 [0.75       0.         0.83333333 0.44444444]
 [0.8        0.33333333 0.88235294 0.7254902 ]
 [0.75       0.66666667 0.76470588 0.73529412]
 [0.75       0.         0.88235294 0.44117647]
 [0.85       1.         0.82352941 0.94117647]
 [0.8        0.         0.88888889 0.38888889]
 [0.8        0.         0.94117647 0.47058824]
 [0.8        0.33333333 0.88235294 0.35294118]
 [0.7        0.         0.82352941 0.29411765]
 [0.65       0.33333333 0.70588235 0.54901961]
 [0.75       0.5        0.77777778 0.72222222]
 [0.95       0.66666667 1.         0.96078431]
 [0.85       

[[0.75       0.         0.88235294 0.70588235]
 [0.75       0.         0.88235294 0.25490196]
 [0.75       0.         0.88235294 0.33333333]
 [0.85       1.         0.82352941 0.88235294]
 [0.9        0.5        0.94444444 0.55555556]
 [0.9        0.33333333 1.         0.58823529]
 [0.85       0.         1.         0.8627451 ]
 [0.75       0.33333333 0.82352941 0.47058824]
 [0.65       0.33333333 0.70588235 0.56862745]
 [0.75       0.5        0.77777778 0.58333333]
 [0.7        0.33333333 0.76470588 0.66666667]
 [0.9        0.33333333 1.         0.8627451 ]
 [0.75       0.66666667 0.76470588 0.66666667]
 [0.9        0.33333333 1.         0.68627451]
 [0.75       0.         0.83333333 0.5       ]
 [0.7        0.66666667 0.70588235 0.70588235]
 [0.75       0.         0.88235294 0.60784314]
 [0.65       0.33333333 0.70588235 0.45098039]
 [0.85       0.66666667 0.88235294 0.75490196]
 [0.85       0.         0.94444444 0.63888889]
 [0.9        0.33333333 1.         0.56862745]
 [0.75       

# Plot ROC curve k-fold CV

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from scipy import interp
from sklearn.metrics import *
def do_cross_validation(X, y, model, fs_model=None, num_folds=10, random_state = 0,  n_runs = 1, verbose=False):
   
    res = np.zeros(shape=(num_folds * n_runs, 9)) # 5 is the number of metrics

    base_fpr = np.linspace(0, 1, 101)   #TODO: add a new parameter for 51
    tprs = []
    count = 0
    for cv_run in range(n_runs):
        skf = StratifiedKFold(n_splits=num_folds, random_state=cv_run, shuffle=True)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            if (fs_model is not None):
                fs_model = fs_model.fit(X_train, y_train)
                train_filtered = fs_model.transform(X_train)
                test_filtered =  fs_model.transform(X_test)
            else:
                train_filtered = X_train
                test_filtered = X_test

            model.fit(train_filtered, y_train)
            Y_probs = model.predict_proba(test_filtered)[:, 1]
            Y_pred = model.predict(test_filtered)
            
            res[count, :] = evaluate2(Y_pred, y_test, Y_probs)

            fpr, tpr, _ = roc_curve(y_test, Y_probs)
            tpr = interp(base_fpr, fpr, tpr)
            tpr[0] = 0.0
            tprs.append(tpr)
            count += 1
    if (verbose):
        print(res)
        print(np.mean(res, axis=0))

    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    rVal = {}
    rVal['metrics'] = np.mean(res, axis=0)
    rVal['metrics_std'] = np.std(res, axis=0)
    rVal['fpr'] = base_fpr
    rVal['tpr'] = mean_tprs

    return rVal

In [ ]:

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Performance reported using:\t ACC\tSn\tSp\tMCC\tAUC')
    for model in models:
        plt.figure(figsize=(12, 12))
        plt.xlabel('False Positive Rate', fontsize='x-large')
        plt.ylabel('True Positive Rate', fontsize='x-large')

        for markers in markers_sets:
            print('{0}: {1}'.format(markers[0], markers[1]))


            print('Shape of training data is {}'.format(np.shape(X)))

            num_runs = 5
            res = do_cross_validation(X, new_y, model=model[1], num_folds=10, fs_model=None, n_runs=num_runs)
            print('{0}: {1}'.format(model[0], res['metrics']))
            plt.plot(res['fpr'], res['tpr'], label=markers[0]+', AUC='+str(round(res['metrics'][4],2)), linewidth=2)

        leg = plt.legend(loc='best', fontsize='x-large')
        plt.suptitle(model[0])
        print('----------------------------------------------------------------------------------------------------')
        plt.show()



# Plot a specific gene marker set and CV

In [ ]:
markers_sets =[('top_15',features[:15])]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Performance reported using:\t ACC\tSn\tSp\tMCC\tAUC')
    for model in models:
        plt.figure(figsize=(12, 12))
        plt.xlabel('False Positive Rate', fontsize='x-large')
        plt.ylabel('True Positive Rate', fontsize='x-large')

        for markers in markers_sets:
            print('{0}: {1}'.format(markers[0], markers[1]))


            print('Shape of training data is {}'.format(np.shape(X)))

            res = do_cross_validation(X, new_y, model=model[1], num_folds=10, fs_model=None, n_runs=num_runs)
            print('{0}: {1}'.format(model[0], res['metrics']))
            plt.plot(res['fpr'], res['tpr'], label=markers[0]+', AUC='+str(round(res['metrics'][4],2)), linewidth=2)

        leg = plt.legend(loc='best', fontsize='x-large')
        plt.suptitle(model[0])
        print('----------------------------------------------------------------------------------------------------')
        plt.show()

In [ ]:
x_ = X_train[:, (combination_value[15])['com']]
x_test = X_test[:, (combination_value[15])['com']]
y_test

# Validation gene marker set

In [ ]:
x_ = X_train[:, (combination_value[15])['com']]
x_test = X_test[:, (combination_value[15])['com']]
# SVM with rbf kernel
plt.figure(0).clf()
models = [
    {
        'label': 'Logistic Regression',
        'model': LogisticRegression(penalty="l2", random_state=0),
    },
    {
        'label': 'XGB',
        'model': XGBClassifier(objective="binary:logistic", n_estimators=100, n_jobs=4, random_state=seed),
    },
    {
        'label' : 'SVM Linear',
        'model': SVC(kernel='rbf',probability=True),
    },
    {
        'label' : 'KNN-15 nei',
        'model': RandomForestClassifier(n_estimators=100,  random_state=seed),
    },
]
for m in models:
    model = m['model'] # select the model
    model.fit(x_, y_train) # train the model
    y_pred=model.predict(x_test) # predict the test data
# Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
# Calculate Area under the curve to display on the plot
    auc = metrics.auc(fpr,tpr)
#     evaluate2(y_pred, y_test, y_probs)
    score2 = evaluate2(y_pred, y_test, model.predict_proba(x_test)[:,1])
# Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (AUC = %0.2f)' % (m['label'], auc))

    print("%s: \nEvaluate score:\nACC\tSn\tSp\tMCC\tAUC\n" % (m['label']), score2)
# # Custom settings for the plot 

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")